## Folium 라이브러리 - 지도 활용

: Folium 라이브러리는 지도 위에 시각화할 때 유용한 도구이다. 세계 지도를 기본 지원하고 다양한 스타일의 지도 이미지를 제공하고 있다.

### 라이브러리 설치

In [1]:
!pip install -forge folium

Looking in links: orge

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip


### 지도 만들기

: Folium 라이브러리의 Map() 함수를 이용하면 지도 객체를 만들 수 있다.지도 화면은 고정된 것이 아니고 zoom 기능과 scroll 기능이 모두 가능하다.    
예제코드를 스파이더와 같은 IDE에서 실행해도 지도가 표시되지 않는다. 이유는 해당 라이브러리는 웹기반 지도를 만들기 때문에 오직 웹 환경에서만 지도를 확인 할 수 있다. 따라서 지도를 보려면 지도 객체에 save() 메소드를 적용하여 html파일로 저장하고 웹브라우저에서 파일을 열어서 확인해야한다.    
쥬피터노트북과 랩은 웹기반의 IDE 이므로 바로 지도객체를 확인할 수 있다.

Map() 함수 옵션에 [위도, 경도] 수치를 입력하면 그지점을 중심으로 지도를 보여준다. zoom_start 옵션을 사용하면 화면 확대 비율을 조절할 수 있다.

In [4]:
# 라이브러리 불러오기
import folium

# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)
seoul_map
# 지도를 HTML 파일로 저장하기
# seoul_map.save('./seoul.html')

### 지도 스타일 적용하기

In [8]:
# 라이브러리 불러오기
import folium

# 서울 지도 만들기
seoul_map2 = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12) # 산악지형 등의 지형이 보다 선명하게 드러난다.
seoul_map2

In [7]:
seoul_map3 = folium.Map(location=[37.55,126.98], tiles='Stamen Toner', 
                        zoom_start=15) # 흑백 스타일로 도로망을 강조해서 보여준다.
seoul_map3

### 지도에 마커 표시하기

- 서울 시내 주요 대학교의 위치 데이터를 데이터 프레임으로 변환하고, 지도에 위치를 표시해보자. 마커 위치를 표시하려면 Marker() 함수에 위도, 경도 정보를 전달한다. popup 옵션을 추가하면 마커를 클릭했을때 팝업창에 표시해주는 텍스트를 넣을수 있다.

In [15]:
# 라이브러리 불러오기
import pandas as pd
import folium

# 대학교 리스트를 데이터프레임 변환
df = pd.read_excel('./서울지역 대학교 위치.xlsx', engine= 'openpyxl', index_col=0)

# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 Marker로 표시
for name, lat, lng in zip(df.index, df.위도, df.경도):
    folium.Marker([lat, lng], popup=name).add_to(seoul_map)
    
seoul_map

이번에는 원형 마커를 표시해 보자. Marker()함수 대신에 CircleMarker() 함수를 사용한다. 원형 마커의 크기, 색상, 투명도등을 설정할 수 있다.

In [16]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.index, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=10,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)
seoul_map

### 지도영역에 단계 구분도(Choropleth Map) 표시하기

- 행정 구역과 같이 지도 상에의 어떤 경계에 둘러싸인 영역에 색을 칠하거나 음영등으로 정보를 나타내는 시각화 방법이다.
- 전달하려는 정보의 값이 커지면 영역에 칠해진 색이나 음영이 진해진다.
- 예제에는 경기도 지역의 시군구별 인구 변화 데이터(2007-2017) , 경기도 행정구역 경계 지리정보를 사용한다.
- Choropleth() 함수를 이용한다.

In [20]:
import pandas as pd
import folium
import json

# 경기도 인구 변화 데이터를 불러와서 데이터프레임으로 변환
filepath = './경기도인구데이터.xlsx'
df = pd.read_excel(filepath, index_col='구분', engine='openpyxl')
df.columns = df.columns.map(str)

# 경기도 시군구 경계 정보를 가진 geo-json 파일 불러오기
geo_path = './경기도행정구역경계.json'
try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig')) #'utf-8-sig'에서 'sig'는 'signature'의 약칭입니다. 'utf-8-sig'를 사용하면 스트링 즉, 문자열로 처리하는게 아니라 Byte Order Mark(BOM)으로 취급

# 경기도 지도 만들기
g_map = folium.Map(location=[37.5502,126.982],
                   titles='Stamen Terrain', zoom_start=9)

# 출력할 연도 선택 (2007~2017)
year = '2017'

# Choropleth 클래스로 단계구분도 표시
folium.Choropleth(geo_data=geo_data,
                  data=df[year],
                  columns= [df.index, df[year]],
                  fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
                 threshold_scale=[10000, 100000, 300000, 500000, 700000],               
                 key_on='feature.properties.name',
                 ).add_to(g_map)
g_map

In [32]:
#year을 바꿔가면서 확인이 가능하다.
# 경기도 인구 변화 데이터를 불러와서 데이터프레임으로 변환
filepath = './경기도인구데이터.xlsx'
df = pd.read_excel(filepath, index_col='구분', engine='openpyxl')
df.columns = df.columns.map(str)

# 경기도 시군구 경계 정보를 가진 geo-json 파일 불러오기
geo_path = './경기도행정구역경계.json'
try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig')) #'utf-8-sig'에서 'sig'는 'signature'의 약칭입니다. 'utf-8-sig'를 사용하면 스트링 즉, 문자열로 처리하는게 아니라 Byte Order Mark(BOM)으로 취급

# 경기도 지도 만들기
g_map = folium.Map(location=[37.5502,126.982],
                   titles='Stamen Terrain', zoom_start=9)

# 출력할 연도 선택 (2007~2017)
year = '2013'

# Choropleth 클래스로 단계구분도 표시
folium.Choropleth(geo_data=geo_data,
                  data=df[year],
                  columns= [df.index, df[year]], 
                  fill_color='YlOrRd', fill_opacity=0.5, line_opacity=0.8, #fill_opacity 색선명도, line_opacity라인선명도
                 threshold_scale=[10000, 100000, 300000, 500000, 700000], # 인원의 정도를 지정              
                 key_on='feature.properties.name',
                 ).add_to(g_map)
g_map

```
folium.Choropleth(
    geo_data = "지도 데이터 파일 경로 (.geojson, geopandas.DataFrame)"
    data = "시각화 하고자 하는 데이터파일. (pandas.DataFrame)"
    columns = (지도 데이터와 매핑할 값, 시각화 하고자하는 변수),
    key_on = "feature.데이터 파일과 매핑할 값",
    fill_color = "시각화에 쓰일 색상",
    legend_name = "칼라 범주 이름",
).add_to(m)
```

파라미터의 내용을 정리한 것이다.